In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import gc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import pickle

import matplotlib.pyplot as plt

# from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline
# from sklearn.tree import plot_tree

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
input_files = []

for dirname, _, filenames in os.walk('dataset\downsampled'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        input_files.append(os.path.join(dirname, filename))
        
print(input_files)
print("\nImports done")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Cleanup for CSECICIDS2018_improved and CICIDS2017_improved

In [ ]:
def label_mapper(label):
    # BENIGN, web_attack, ddos, botnet, infiltration, dos, ssh_bruteforce, ftp_bruteforce, others
    if label == 'BENIGN':
        return 0
    elif label.startswith("Web Attack"):
        return 1
    elif label.startswith("DDoS"):
        return 2
    elif label.startswith("Botnet"):
        return 3
    elif label.startswith("Infiltration") or label.startswith("Portscan") :
        return 4
    elif label.startswith("DoS"):
        return 5
    elif label.startswith("SSH"):
        return 6
    elif label.startswith("FTP"):
        return 7
    else:
        return 8


def dataset_cleaner(df):
    # change label
    df['Label'] = df['Label'].apply(label_mapper)
    
    # drop all categorical columns
    categorical_columns = df.select_dtypes(include=['object'])
    df.drop(columns=categorical_columns.columns.values, inplace=True)
    # id is unique for all, CISIAS notebook removed attempted category too
    df.drop(columns=['id', 'Attempted Category'], inplace=True)
    
#     print(df.columns.values)
    
    # drop all inf and other nan values
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(axis=0,how='any', inplace=True)
    
    return df

def get_target_features(df, test_size = 0.30):
    X = df.drop(['Label'], axis=1)
    y = df['Label']
    
    return train_test_split(X, y, test_size=test_size, random_state=42)

print("Functions defined")

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif

def select_features_correlation(X, y, k=10):
    """
    Select top k features based on correlation with the target.

    Parameters:
    - X: DataFrame, the feature matrix.
    - y: Series, the target variable.
    - k: int, number of top features to select.

    Returns:
    - List of top k features based on correlation.
    """
    # Factorize the target and convert to pandas Series
    y_factorized = pd.Series(y.factorize()[0], index=y.index)
    
    # Convert categorical features to numeric
    X_encoded = X.apply(LabelEncoder().fit_transform)
    
    # Compute correlation
    correlation = X_encoded.corrwith(y_factorized)
    
    # Create a DataFrame for correlation values
    correlation_df = pd.DataFrame({
        'Feature': correlation.index,
        'Correlation': correlation.values
    }).sort_values(by='Correlation', key=abs, ascending=False)
    
    # Select top k features
    return correlation_df.head(k)['Feature'].tolist()

def select_features_information_gain(X, y, k=10):
    """
    Select top k features based on information gain with the target.

    Parameters:
    - X: DataFrame, the feature matrix.
    - y: Series, the target variable.
    - k: int, number of top features to select.

    Returns:
    - List of top k features based on information gain.
    """
    # Convert categorical features to numeric
    X_encoded = X.apply(LabelEncoder().fit_transform)
    
    # Calculate information gain
    info_gain = mutual_info_classif(X_encoded, y, discrete_features='auto', random_state=42)
    
    # Create a DataFrame for information gain values
    info_gain_df = pd.DataFrame({
        'Feature': X.columns,
        'Information Gain': info_gain
    }).sort_values(by='Information Gain', ascending=False)
    
    # Select top k features
    return info_gain_df.head(k)['Feature'].tolist()

def get_top_features_across_files(file_paths, label_column, k=10):
    """
    Get top k features across multiple files, based on both correlation and information gain.

    Parameters:
    - file_paths: list of file paths.
    - label_column: name of the label column.
    - k: number of top features to select.

    Returns:
    - List of top k features consistent across all files.
    """
    feature_counter = Counter()
    idx = 1

    for file_path in file_paths:
        df = pd.read_csv(file_path)
        X = df.drop(columns=[label_column])
        y = df[label_column]
        
        # Select top features using both methods
        top_corr_features = select_features_correlation(X, y, k)
        top_ig_features = select_features_information_gain(X, y, k)
        
        # Combine and count features
        feature_counter.update(top_corr_features + top_ig_features)
        print("File "+str(idx)+" done.")
        idx = idx+1
    
    # Get most common features across files
    most_common_features = feature_counter.most_common(k)
    return [feature for feature, _ in most_common_features]

def remove_selected_features(df, selected_features):
    """
    Remove selected features from the given DataFrame.

    Parameters:
    - df: DataFrame, the input DataFrame.
    - selected_features: list of feature names to be removed.

    Returns:
    - DataFrame with the selected features removed.
    """
    # Drop the selected features
    updated_df = df.drop(columns=selected_features, errors='ignore')
    return updated_df


In [ ]:
top_features = get_top_features_across_files(input_files,'Label',20)
print(top_features)

# Load each file, do label clean up, run KNN on it

In [ ]:
# we will add to these 3 lists test set and predictions for all csv
# then we will merge them in one pandas object
# then we can do metrics on the dataset as a whole
# pros: will get whole dataset metrics
# cons: the 3 list and merged object may overflow 30 gb ram (praying it wont)
X_test_sets = []
y_test_sets = []

# MODELS
print("Initializing KNN model")
knn_model = KNeighborsClassifier(n_neighbors=3)

i = 0

for f in input_files:
    
    i += 1
        
    print(f"{i}/{len(input_files)} CURRENT FILE: {f}\n")
    
    df = pd.read_csv(f)
    
    print(f"LOADED FILE {f}, STARTING CLEANUP\n")
    
    df = dataset_cleaner(df)

    
    df = remove_selected_features(df,top_features)
    
    print(f"CLEANUP DONE, STARTING SPLIT\n")
    
    X_train, X_test, y_train, y_test = get_target_features(df, 0.30)
    
    # no need for df now
    df = None
    gc.collect()
    
    X_test.to_csv(f"test_chunks/{i}X_test.csv", index=False)
    y_test.to_csv(f"test_chunks/{i}y_test.csv", index=False)
    
    # no need for the test sets either
    del X_test
    del y_test
    gc.collect()
    
    print("DONE WRITING TEST SETS TO FILES, STARTING TRAIN\n")
    
    # KNN
    print("RUNNING KNN MODEL TRAIN...\n")
    
    knn_model.fit(X_train, y_train)
    
    del X_train
    del y_train
    gc.collect()
    
    print(f"KNN MODEL TRAINED ON FILE: {f} \n\n")
    

print(f"{i}/{len(input_files)} files worked on\n\n")

# clear memory
df = None
gc.collect()

# Predict and prepare outputs for metric calculation

In [ ]:
print("READING X TEST SETS AND PREDICTING\n")
gc.collect()

knn_predictions = []
y_tests = []

for curr in range(1,i+1):
    print(f"Reading X_test file {curr}/{i}\n")

    df_X = pd.read_csv(f"test_chunks/{curr}X_test.csv")
    
    print(f"Read {curr}-th X_test file, shape: {df_X.shape}\nPredicting and appending...")
    
    pred = knn_model.predict(df_X)
    knn_predictions.append(pred)
    
    del df_X 
    gc.collect()
    
    print(f"Done reading and predictiong on X_test file {curr}/{i}\n")
    

knn_predictions_joined = np.concatenate(knn_predictions)

# remove prediction list
del knn_predictions
gc.collect()

print("PREDICTION DONE USING KNN MODELS: ")
print( knn_predictions_joined.shape)

print("\nREADING y_test FILES\n")

for curr in range(1,i+1):
    print(f"Reading y_test file {curr}/{i}\n")

    df_y = pd.read_csv(f"test_chunks/{curr}y_test.csv")
    
    print(f"Read {curr}-th y_test file, shape: {df_y.shape}\nAppending...")
    
    y_tests.append(df_y)
    
    del df_y
    gc.collect()
    
    print(f"Done reading and adding y_test file {curr}/{i}\n")
    
y_test_sets_joined = pd.concat(y_tests)

# remove y_tests
del y_tests
gc.collect()


print("DONE READING y_test FILES: ")
print(y_test_sets_joined.shape)


# Metrics

In [ ]:
knn_accuracy = accuracy_score(y_test_sets_joined, knn_predictions_joined)
knn_f1 = f1_score(y_test_sets_joined, knn_predictions_joined, average='macro')
knn_precision = precision_score(y_test_sets_joined, knn_predictions_joined, average='macro')
knn_recall = recall_score(y_test_sets_joined, knn_predictions_joined, average='macro')

print("KNN METRICS: \n")
print(f"ACCURACY: \t {knn_accuracy:.4f}")
print(f"F1 SCORE: \t {knn_f1:.4f}")
print(f"PRECISION: \t {knn_precision:.4f}")
print(f"RECALL: \t {knn_recall:.4f}")

# Plots

In [ ]:
# Confusion Matrix
classes = ["BENIGN", "Web Attack", "DDoS", "Botnet", "Infiltration", "DoS", "SSH Bruteforce", "FTP Bruteforce", "Others"]
title = "Confusion Matrix for KNN Classifier"

cm = confusion_matrix(y_test_sets_joined, knn_predictions_joined)
plt.figure(figsize=(14, 14))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.title(title)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
def save_model(model, filename="model.pkl"):
    with open(filename, "wb") as file:
        pickle.dump(model, file)
    print(f"Model saved to {filename}")

In [ ]:
def load_model(filename="model.pkl"):
    with open(filename, "rb") as file:
        model = pickle.load(file)
    print(f"Model loaded from {filename}")
    return model

In [ ]:
# Save to a pickle file            
save_model(knn_model,"models/knn.pickle")